# Đồ án cuối kì (Lập trình cho Khoa học dữ liệu)

Giao vien
Thông tin nhóm:
1. Nguyễn Văn Hậu (1712258)
2. Hà Thị Ngọc Thăm (1712160)

Phân công công việc: https://docs.google.com/spreadsheets/d/1yk7EGQSebSC8R5h_UnNOg1wxUU1y9y6lNY5y9waDXA4/edit?usp=sharing

Github: 

In [ ]:
import numpy as np
import pandas as pd

# Thu thập dữ liệu:

- Tên tập dữ liệu: **"Medium 2021 data science articles dataset"**
- Mô tả về dữ liệu:
    - RAW: "Data about 45k+ data science articles published on medium in 2021."
    - Vietsub: Dữ liệu khoảng trên 45 ngàn dòng về các bài báo khoa học trên Medium vào năm 2021.
    - Tập dữ liệu tập trung vào các lĩnh vực:  Data Science, Machine Learning, Artificial Inteligence, Deep Learning, Data, Big Data, Analytics.
- Tác giả: Vinicius Lambert
- Nguồn dataset: [Kaggle](https://www.kaggle.com/viniciuslambert/medium-2021-data-science-articles-dataset)
- License: **[CC0: Public Domain](https://creativecommons.org/publicdomain/zero/1.0/)** (No Copyright)
- Tác giả thu thập data bằng cách parser HTML, được trình bày cách thu thập tại [file notebook](https://github.com/viniciusLambert/medium-scraping/blob/main/scrap.ipynb) này.

# Khám phá dữ liệu:

Thử xem những dòng đầu tiên của data có hình dạng ra sao.

In [ ]:
medium_data = pd.read_csv("Data/medium-data-science-articles-2021.csv")
medium_data.head()

,url,title,author,author_page,subtitle,claps,responses,reading_time,tag,date
0,https://medium.com/@dharmeshpanchmatia/data-an...,Data Analytics and AI/ML platform for eCommerce,dharmeshpanchmatia,https://medium.com/@dharmeshpanchmatia,Improve user pr,30.0,0.0,5.0,Big Data,2021-01-01
1,https://medium.com/predict/the-journey-from-an...,The journey from an Apple Silicon M1 powered l...,dimitrisv,https://medium.com/@dimitrisv,NaN,42.0,0.0,5.0,Big Data,2021-01-01
2,https://medium.com/@rajsaraogi/demystifying-kp...,Demystifying KPIs in Analytics,rajsaraogi,https://medium.com/@rajsaraogi,NaN,19.0,0.0,3.0,Big Data,2021-01-01
3,https://medium.com/@bigdataschool/%D0%B1%D0%BE...,Большая разница: чем структурированная потоков...,bigdataschool,https://medium.com/@bigdataschool,NaN,0.0,0.0,6.0,Big Data,2021-01-01
4,https://medium.com/@arnowa-44509/iot-a-new-way...,IoT: A New Way of Life,arnowa-44509,https://medium.com/@arnowa-44509,"IoT is more than just the Internet of things, ...",0.0,0.0,3.0,Big Data,2021-01-01


## Dữ liệu có bao nhiêu dòng và bao nhiêu cột?

- `num_cols` là số cột 
- `num_rows` là số dòng 

# Đồ án cuối kì (Lập trình cho Khoa học dữ liệu)

Giao vien
Thông tin nhóm:
1. Nguyễn Văn Hậu (1712258)
2. Hà Thị Ngọc Thăm (1712160)

Phân công công việc: https://docs.google.com/spreadsheets/d/1yk7EGQSebSC8R5h_UnNOg1wxUU1y9y6lNY5y9waDXA4/edit?usp=sharing

Github: 

In [ ]:
import numpy as np
import pandas as pd

# Thu thập dữ liệu:

- Tên tập dữ liệu: **"Medium 2021 data science articles dataset"**
- Mô tả về dữ liệu:
    - RAW: "Data about 45k+ data science articles published on medium in 2021."
    - Vietsub: Dữ liệu khoảng trên 45 ngàn dòng về các bài báo khoa học trên Medium vào năm 2021.
    - Tập dữ liệu tập trung vào các lĩnh vực:  Data Science, Machine Learning, Artificial Inteligence, Deep Learning, Data, Big Data, Analytics.
- Tác giả: Vinicius Lambert
- Nguồn dataset: [Kaggle](https://www.kaggle.com/viniciuslambert/medium-2021-data-science-articles-dataset)
- License: **[CC0: Public Domain](https://creativecommons.org/publicdomain/zero/1.0/)** (No Copyright)
- Tác giả thu thập data bằng cách parser HTML, được trình bày cách thu thập tại [file notebook](https://github.com/viniciusLambert/medium-scraping/blob/main/scrap.ipynb) này.

# Khám phá dữ liệu:

Thử xem những dòng đầu tiên của data có hình dạng ra sao.

In [ ]:
medium_data = pd.read_csv("Data/medium-data-science-articles-2021.csv")
medium_data.head()

,url,title,author,author_page,subtitle,claps,responses,reading_time,tag,date
0,https://medium.com/@dharmeshpanchmatia/data-an...,Data Analytics and AI/ML platform for eCommerce,dharmeshpanchmatia,https://medium.com/@dharmeshpanchmatia,Improve user pr,30.0,0.0,5.0,Big Data,2021-01-01
1,https://medium.com/predict/the-journey-from-an...,The journey from an Apple Silicon M1 powered l...,dimitrisv,https://medium.com/@dimitrisv,NaN,42.0,0.0,5.0,Big Data,2021-01-01
2,https://medium.com/@rajsaraogi/demystifying-kp...,Demystifying KPIs in Analytics,rajsaraogi,https://medium.com/@rajsaraogi,NaN,19.0,0.0,3.0,Big Data,2021-01-01
3,https://medium.com/@bigdataschool/%D0%B1%D0%BE...,Большая разница: чем структурированная потоков...,bigdataschool,https://medium.com/@bigdataschool,NaN,0.0,0.0,6.0,Big Data,2021-01-01
4,https://medium.com/@arnowa-44509/iot-a-new-way...,IoT: A New Way of Life,arnowa-44509,https://medium.com/@arnowa-44509,"IoT is more than just the Internet of things, ...",0.0,0.0,3.0,Big Data,2021-01-01


## Dữ liệu có bao nhiêu dòng và bao nhiêu cột?

- `num_cols` là số cột 
- `num_rows` là số dòng 

In [ ]:
num_rows, num_cols = medium_data.shape
print("So cot la: {}, so dong la: {}".format(num_cols, num_rows))

So cot la: 10, so dong la: 47660


Đúng theo mô tả về data của tác giả.

## Mỗi dòng có ý nghĩa gì? Các dòng có ý nghĩa khác nhau không?

Theo file mô tả dữ liệu và theo quan sát sơ bộ về dữ liệu thì một dòng cho biết thông tin chi tiết của một bài báo bao gồm: url, title, author,..... Có vẻ không có dòng nào bị "lạc loài".

## Dữ liệu có các dòng bị lặp không?

In [ ]:
num_duplicated_rows = medium_data.index.duplicated().sum()
num_duplicated_rows

0

Dữ liệu không có dòng nào bị lặp.

## Ý nghĩa mỗi cột?

Mô tả ý nghĩa mỗi cột của dữ liệu:
- Url: địa chỉ của bài báo trên web.
- Title: tiêu đề của bài báo.
- Author: tác giả của bài báo.
- Author_page: địa chỉ trang của tác giả trên web.
- Subtitle: phụ đề của bài báo.
- Claps: số lượt vỗ tay của bạn đọc.
- Responses: số lượt phản hồi từ bạn đọc.
- Reading_time: thời gian đọc đề xuất cho người xem. 
- Tag: chủ đề của bài báo.
- Date: ngày xuất bản.

## Kiểm tra kiểu dữ liệu của cột:

Mỗi cột hiện đang thuộc kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp đề có thể xử lý tiếp không?

## Kiểu dữ liệu dạng số (numeric):

Các giá trị được phân bố như thế nào? (Số lượng/ Tỉ lệ các giá trị thiếu?/Min?/Max? -> Kiểm tra bất thường?)

## Kiểu dữ liệu dạng phân loại (categorical):

các giá trị phân bố như thế nào? (Số lượng/ Tỉ lệ các giá trị thiếu?/ Số lượng các giá trị khác nhau? Show một vài giá trị?-> Có gì bất thường hay không?)

# Đặt câu hỏi

Yêu cầu:
- Câu hỏi là gì? Cần có ý nghĩa?
- Nếu trả lời được câu hỏi thì sẽ có lợi ích gì?
- Không nên quá dễ để trả lời?


Với mỗi câu hỏi:
- Cần có tiền xử lý dữ liệu không? Và nếu có thì nhóm bạn xử lý như thế nào? (Gồm text mô tả và code bên dưới)
- Nhóm bạn phân tích dữ liệu như thế nào để ra được câu trả lời cho câu hỏi? (text và code)

## Câu hỏi số 1: (Haauj)

- Mô tả chi tiết:
- Ý nghĩa của câu hỏi:

## Câu hỏi số 2: (Thawms)

- Mô tả chi tiết:
- Ý nghĩa của câu hỏi:

# Nhìn lại quá trình làm đồ án:

- Mỗi thành viên đã gặp những khó khăn gì?
- Mồi thành viên học được gì hữu ích.
- Nhóm: Nếu có thêm thời gian thì sẽ làm gì?

Hau

Tham

Nhóm:

# Tài liệu tham khảo:

- Tài liệu môn học
- Dataset: [Medium 2021 data science articles dataset](https://www.kaggle.com/viniciuslambert/medium-2021-data-science-articles-dataset), Vinicius Lambert.

In [ ]:
num_rows, num_cols = medium_data.shape
print("So cot la: {}, so dong la: {}".format(num_cols, num_rows))

So cot la: 10, so dong la: 47660


Đúng theo mô tả về data của tác giả.

## Mỗi dòng có ý nghĩa gì? Các dòng có ý nghĩa khác nhau không?

Theo file mô tả dữ liệu và theo quan sát sơ bộ về dữ liệu thì một dòng cho biết thông tin chi tiết của một bài báo bao gồm: url, title, author,..... Có vẻ không có dòng nào bị "lạc loài".

## Dữ liệu có các dòng bị lặp không?

In [ ]:
num_duplicated_rows = medium_data.index.duplicated().sum()
num_duplicated_rows

0

Dữ liệu không có dòng nào bị lặp.

## Ý nghĩa mỗi cột?

Mô tả ý nghĩa mỗi cột của dữ liệu:
- Url: địa chỉ của bài báo trên web.
- Title: tiêu đề của bài báo.
- Author: tác giả của bài báo.
- Author_page: địa chỉ trang của tác giả trên web.
- Subtitle: phụ đề của bài báo.
- Claps: số lượt vỗ tay của bạn đọc.
- Responses: số lượt phản hồi từ bạn đọc.
- Reading_time: thời gian đọc đề xuất cho người xem. 
- Tag: chủ đề của bài báo.
- Date: ngày xuất bản.

## Kiểm tra kiểu dữ liệu của cột:

Mỗi cột hiện đang thuộc kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp đề có thể xử lý tiếp không?

## Kiểu dữ liệu dạng số (numeric):

Các giá trị được phân bố như thế nào? (Số lượng/ Tỉ lệ các giá trị thiếu?/Min?/Max? -> Kiểm tra bất thường?)

## Kiểu dữ liệu dạng phân loại (categorical):

các giá trị phân bố như thế nào? (Số lượng/ Tỉ lệ các giá trị thiếu?/ Số lượng các giá trị khác nhau? Show một vài giá trị?-> Có gì bất thường hay không?)

# Đặt câu hỏi

Yêu cầu:
- Câu hỏi là gì? Cần có ý nghĩa?
- Nếu trả lời được câu hỏi thì sẽ có lợi ích gì?
- Không nên quá dễ để trả lời?


Với mỗi câu hỏi:
- Cần có tiền xử lý dữ liệu không? Và nếu có thì nhóm bạn xử lý như thế nào? (Gồm text mô tả và code bên dưới)
- Nhóm bạn phân tích dữ liệu như thế nào để ra được câu trả lời cho câu hỏi? (text và code)

## Câu hỏi số 1: (Haauj)

- Mô tả chi tiết:
- Ý nghĩa của câu hỏi:

## Câu hỏi số 2: (Thawms)

- Mô tả chi tiết:
- Ý nghĩa của câu hỏi:

# Nhìn lại quá trình làm đồ án:

- Mỗi thành viên đã gặp những khó khăn gì?
- Mồi thành viên học được gì hữu ích.
- Nhóm: Nếu có thêm thời gian thì sẽ làm gì?

Hau

Tham

Nhóm:

# Tài liệu tham khảo:

- Tài liệu môn học
- Dataset: [Medium 2021 data science articles dataset](https://www.kaggle.com/viniciuslambert/medium-2021-data-science-articles-dataset), Vinicius Lambert.